<a href="https://colab.research.google.com/github/18918852785/ctptrader/blob/master/%E2%80%9Ctslearn_explore_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install netdata-pandas==0.0.15 tslearn==0.4.1 am4894plots==0.2.0

In [ ]:
import pandas as pd
import numpy as np
import textwrap
from scipy.fftpack import rfft
from scipy.stats import boxcox
from sklearn.cluster import DBSCAN, AgglomerativeClustering, Birch, MeanShift, AffinityPropagation, OPTICS
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans, KShape, KernelKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from netdata_pandas.data import get_data, get_chart_list
from am4894plots.plots import plot_lines, plot_lines_grid, plot_scatters

In [ ]:
# inputs
# host running netdata that we want to pull data from, list here: https://learn.netdata.cloud/docs/agent/demo-sites
host = np.random.choice(['odyslam.me/netdata/webserver', 'london.my-netdata.io', 'octopuscs.my-netdata.io', 'cdn77.my-netdata.io'])
print(host)
# randomly sample a length of time in last 60 minutes
lengths = [3, 5, 7, 10, 15, 20]
lengths = [3, 5]
after = np.random.randint(min(lengths), 60) * -1 * 60
length = np.random.choice(lengths) * 60
before = after + length
n = after - before
print(after, before, n)
n_charts = None # If None then pull data for all charts otherwise sample n_charts randomly
charts = None #['system.cpu', 'system.load'] # if want to select specific charts
n_clusters = 50 # number of clusters to fit
diff = False # take diffs of the data or not
preprocessing_meanvar = False # True to use TimeSeriesScalerMeanVariance preprocessing
smooth_n = 15 # n observations to smooth over
smooth_func = 'mean' # one of ['mean','min','max','sum', 'median']
norm = True # normalize the data to 0-1 range
model = 'kmeans' # one of ['kmeans','kshape','kernelkmeans','dtw','softdtw','dbscan','agglomerative']
min_n = 4 # only interested in clusters with min_n or more members
preprocessing_fft = False # True if you want to do the clustering based on fft transformation of X
preprocessing_sqrt = True
preprocessing_log = False
dim_red = 'pca' # 'tsne' or 'pca'
cluster_quality_method = 'sil' # 'sil' or 'corr'

octopuscs.my-netdata.io
-2820 -2520 -300


In [ ]:
# get charts
if charts:
    print(charts)
elif n_charts:
    charts = np.random.choice(get_chart_list(host), n_charts).tolist()
    print(charts)
else:
    charts = get_chart_list(host)
# get data
df = get_data(host, charts, after=after, before=before)
print(df.shape)
# remove duplicate columns that we might get from get_data()
df = df.loc[:,~df.columns.duplicated()]
# drop any empty columns (it can happen)
df = df.dropna(axis=1, how='all')
# forward fill and backward fill to try remove any N/A values
df = df.ffill().bfill()
# do sqrt
if preprocessing_sqrt:
    df = df.apply(lambda col: np.sqrt(col))
# do log
if preprocessing_log:
    df = df.apply(lambda col: np.log1p(col))
# take differences if specified
if diff:
    df = df.diff()
# do any smoothing as specified
if smooth_n > 0:
    if smooth_func == 'mean':
        df = df.rolling(smooth_n).mean().dropna(how='all')
    elif smooth_func == 'max':
        df = df.rolling(smooth_n).max().dropna(how='all')
    elif smooth_func == 'min':
        df = df.rolling(smooth_n).min().dropna(how='all')
    elif smooth_func == 'sum':
        df = df.rolling(smooth_n).sum().dropna(how='all')
    elif smooth_func == 'median':
        df = df.rolling(smooth_n).median().dropna(how='all')
    else:
        df = df.rolling(smooth_n).mean().dropna(how='all')
# normalize the data if specified
if norm:
    df = (df-df.min())/(df.max()-df.min())
# drop any empty columns that may remain
df = df.dropna(axis=1, how='all')
# set index to be a datetime for better plotting later
df = df.set_index(pd.to_datetime(df.index, unit='s'))
# look at our data
print(df.shape)
print(df.index.min(), df.index.max())
df.head()

(2832, 3917)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

invalid value encountered in sqrt



(2818, 1883)
2020-09-15 13:20:39 2020-09-15 14:07:36


,ipv4.icmp|received,system.ctxt|switches,netdata.plugin_statsd_charting_cpu|user,netdata.plugin_statsd_charting_cpu|system,system.load|load1,system.load|load5,system.load|load15,disk_inodes._|avail,disk_inodes._|used,system.processes|running,system.intr|interrupts,mem.committed|Committed_AS,mem.writeback|Dirty,mem.writeback|Writeback,system.uptime|uptime,mem.ksm_savings|offered,cpu.cpu1|steal,cpu.cpu1|softirq,cpu.cpu1|user,cpu.cpu1|system,cpu.cpu1|nice,cpu.cpu1|iowait,ip.mcast|received,ip.tcpofo|inqueue,ip.tcpconnaborts|baddata,ip.tcpconnaborts|timeout,ipv4.sockstat_udp_sockets|inuse,ipv4.tcpsock|connections,mem.slab|reclaimable,mem.slab|unreclaimable,ipv4.sockstat_sockets|used,ipv4.sockstat_tcp_mem|mem,mem.ksm_ratios|savings,mem.pgfaults|minor,net_packets.vpn0|received,netdata.plugin_statsd_collector1_cpu|user,netdata.plugin_statsd_collector1_cpu|system,netdata.clients|clients,ip.mcastpkts|received,disk_backlog.sda|backlog,...,users.lreads|www-data,users.lreads|192,users.lreads|193,users.lreads|998,users.lreads|195,users.lreads|81,users.lreads|systemd-resolve,users.lreads|997,users.lreads|992,users.lreads|995,users.lreads|492,users.lreads|postfix,users.lreads|systemd-network,users.lreads|499,users.lreads|netdata,users.lreads|root,users.lreads|sshd,users.lreads|51,users.lreads|498,cgroup_lxc_centos7.cpu_per_core|cpu0,cgroup_lxc_centos7.cpu_per_core|cpu1,cgroup_lxc_centos7.cpu_per_core|cpu2,cgroup_lxc_centos7.cpu_per_core|cpu3,services.cpu|dbus,services.cpu|cron,services.cpu|rc-local,services.cpu|postfix,services.cpu|nginx,services.cpu|accounts-daemon,services.cpu|systemd-journald,services.cpu|systemd-timesyncd,services.cpu|ssh,services.cpu|systemd-logind,services.cpu|netdata,services.cpu|systemd-udevd,services.cpu|lxc-net,services.cpu|irqbalance,services.cpu|rsyslog,services.cpu|lxcfs,services.cpu|ulogd2
time_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-15 13:20:39,0.624952,0.144969,0.0,0.0,0.219223,0.699540,0.818924,0.059797,0.940205,0.721320,0.111947,0.212202,0.034339,0.0,0.000000,0.0,0.0,0.248205,0.588509,0.329283,0.808976,0.0,0.499845,0.0,0.0,0.0,0.0,0.873091,0.000000,0.351645,0.838477,0.518284,0.90447,0.102261,0.436306,0.0,0.0,6.242417e-14,0.499844,0.250654,...,0.981923,0.330833,0.024585,0.0,0.0,0.0,0.0,0.372885,0.261590,0.321039,0.291282,0.378092,0.396048,0.255622,0.127749,0.015120,0.335588,0.0,0.400061,0.169925,1.000000,0.417674,0.364100,0.0,0.0,0.810397,0.000000,0.0,0.49874,0.0,0.0,1.000000,0.0,0.397931,0.0,0.0,0.0,0.0,0.513186,0.0
2020-09-15 13:20:40,0.624952,0.128746,0.0,0.0,0.217286,0.698019,0.817789,0.059797,0.940205,0.682366,0.099103,0.213242,0.034461,0.0,0.000317,0.0,0.0,0.248205,0.562742,0.345471,0.802809,0.0,0.790087,0.0,0.0,0.0,0.0,0.915278,0.000000,0.353199,0.842878,0.500648,0.90447,0.125123,0.458862,0.0,0.0,6.242417e-14,0.790083,0.196602,...,0.981923,0.330833,0.024585,0.0,0.0,0.0,0.0,0.363072,0.278504,0.324565,0.288565,0.378092,0.395912,0.255569,0.131911,0.013760,0.335366,0.0,0.376341,0.163007,0.978554,0.407620,0.405459,0.0,0.0,0.752647,0.000000,0.0,0.49874,0.0,0.0,0.961234,0.0,0.400039,0.0,0.0,0.0,0.0,0.535222,0.0
2020-09-15 13:20:41,0.701999,0.141961,0.0,0.0,0.215349,0.696499,0.816654,0.059797,0.940205,0.606815,0.124336,0.214283,0.034583,0.0,0.000659,0.0,0.0,0.185926,0.499811,0.326704,0.808869,0.0,0.999341,0.0,0.0,0.0,0.0,0.957465,0.000000,0.367165,0.849479,0.506152,0.90447,0.136125,0.467166,0.0,0.0,6.242417e-14,0.999340,0.146818,...,0.981923,0.330833,0.024585,0.0,0.0,0.0,0.0,0.361479,0.277786,0.343890,0.289220,0.570408,0.375516,0.254175,0.137609,0.013561,0.280372,0.0,0.374525,0.169121,0.951979,0.407378,0.444544,0.0,0.0,0.694896,0.000000,0.0,0.49874,0.0,0.0,0.946154,0.0,0.405566,0.0,0.0,0.0,0.0,0.547649,0.0
2020-09-15 13:20:42,0.680450,0.330755,0.0,0.0,0.213412,0.694979,0.815519,0.059797,0.940205,0.576555,0.287737,0.218762,0.034705,0.0,0.001025,0.0,0.0,0.248205,0.580513,0.417294,0.766627,0.0,0.999341,0.0,0.0,0.0,0.0,0.985763,0.000000,0.385008,0.852780,0.50047

In [ ]:
# get values to cluster on
if preprocessing_fft:
    X = rfft(df).transpose()
else:
    X = df.transpose().values
if preprocessing_meanvar:
    X = TimeSeriesScalerMeanVariance().fit_transform(X)
    df = pd.DataFrame(X.reshape(df.shape), columns=df.columns, index=df.index)
if model == 'kshape':
    model = KShape(n_clusters=n_clusters, max_iter=10, n_init=2).fit(X)
elif model == 'kmeans':
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=10, n_init=2).fit(X)
elif model == 'dtw':
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=5, n_init=2).fit(X)
elif model == 'softdtw':
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="softdtw", max_iter=5, n_init=2).fit(X)
elif model == 'kernelkmeans':
    model = KernelKMeans(n_clusters=n_clusters, kernel="gak", max_iter=5, n_init=2).fit(X)
elif model == 'dbscan':
    model = DBSCAN().fit(X)
elif model == 'agglomerative':
    model = AgglomerativeClustering(distance_threshold=0.9, n_clusters=None).fit(X)
elif model == 'birch':
    model = Birch(n_clusters=n_clusters).fit(X)
elif model == 'meanshift':
    model = MeanShift().fit(X)
elif model == 'affinity':
    model = AffinityPropagation().fit(X)
elif model == 'optics':
    model = OPTICS().fit(X)
else:
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=10, n_init=2).fit(X)

In [ ]:
print(len(set(model.labels_)))
print(model.labels_)

silhouette_avg = silhouette_score(X, model.labels_)
print(round(silhouette_avg,2))

50
[37 30 14 ... 15 49  1]
0.24


In [ ]:
# build helper df to map metrics to their cluster labels
df_cluster = pd.DataFrame(list(zip(df.columns, model.labels_)), columns=['metric', 'cluster'])

# make some helper dictionaries and lists
cluster_metrics_dict = df_cluster.groupby(['cluster'])['metric'].apply(lambda x: [x for x in x]).to_dict()
cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
clusters_all = [cluster for cluster in cluster_len_dict]
clusters_dropped = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]<min_n]
clusters_final = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]>=min_n]

df_cluster.head()

,metric,cluster
0,ipv4.icmp|received,37
1,system.ctxt|switches,30
2,netdata.plugin_statsd_charting_cpu|user,14
3,netdata.plugin_statsd_charting_cpu|system,1
4,system.load|load1,20


In [ ]:
# get quality score for each cluster
if cluster_quality_method == 'sil':
    silhouette_scores = silhouette_samples(X, model.labels_)
    df_cluster_silhouette_scores = pd.DataFrame(list(zip(model.labels_, silhouette_scores)), columns=['cluster', 'silhouette_score'])
    df_cluster_silhouette_scores = df_cluster_silhouette_scores.groupby(['cluster']).mean()
    cluster_quality_dict = df_cluster_silhouette_scores.to_dict()['silhouette_score']
else:
    cluster_quality_dict = {}
    for cluster_number in clusters_all:
        x_corr = df[cluster_metrics_dict[cluster_number]].corr().abs().values
        x_corr_mean = round(x_corr[np.triu_indices(x_corr.shape[0],1)].mean(),2)
        cluster_quality_dict[cluster_number] = x_corr_mean

In [ ]:
# build cluster level df with some cluster metadata
df_cluster_meta = pd.DataFrame.from_dict(cluster_len_dict, orient='index', columns=['n'])
df_cluster_meta.index.names = ['cluster']
df_cluster_meta['quality_score'] = df_cluster_meta.index.map(cluster_quality_dict)
df_cluster_meta = df_cluster_meta.sort_values('quality_score', ascending=False)
df_cluster_meta.head()

,n,quality_score
cluster,,
48,2,1.000000
28,2,1.000000
21,3,1.000000
26,2,0.999656
0,270,0.841391


In [ ]:
# fit a pca or tsne on the cluster centers
cluster_centers = model.cluster_centers_.reshape(model.cluster_centers_.shape[0],model.cluster_centers_.shape[1])
if dim_red == 'tsne':
    m = TSNE(n_components=2)
    projected = m.fit_transform(cluster_centers)
else:
    m = PCA(n_components=5)
    m.fit(X)
    print(m.explained_variance_ratio_)
    projected = m.transform(cluster_centers)
df_projected = pd.DataFrame(projected[:, 0:2], columns=['pc1','pc2'])
df_projected['cluster'] = range(model.cluster_centers_.shape[0])
df_projected = df_projected.set_index('cluster')

plot_scatters(df_projected, ['pc1','pc2'], renderer='colab', h=600, w=800, n_cols=1, marker_size=8, labels=df_projected.index)

[0.60294129 0.13728216 0.03764183 0.02659572 0.01628979]


In [ ]:
#plot_lines(df_cluster_centers, [0, 30], renderer='colab', theme=None)
#plot_lines(df, cols=cluster_metrics_dict[42], renderer='colab', theme=None)

In [ ]:
# plot each cluster
for cluster_number, row in df_cluster_meta.iterrows():
    if cluster_number in clusters_final:
        plot_title = f"cluster {cluster_number} (quality={row['quality_score']}, n={row['n']})"
        plot_lines(df, cols=cluster_metrics_dict[cluster_number], renderer='colab', theme=None, title=plot_title)

In [ ]:
# print cluster centers
df_cluster_centers = pd.DataFrame(model.cluster_centers_.reshape(model.cluster_centers_.shape[0],model.cluster_centers_.shape[1])).transpose()
df_cluster_centers.index = df.index
for cluster in df_cluster_meta.index:
    print('\n'.join(textwrap.wrap(f"Cluster {cluster} (n={cluster_len_dict[cluster]}, score = {cluster_quality_dict[cluster]}): {','.join(cluster_metrics_dict[cluster])}", width=175, replace_whitespace=False)))
    plot_lines(df_cluster_centers, [cluster], renderer='colab', theme=None, title=f"Cluster {cluster}")
